# Recommendation System for Amazon Clothing Products
---
## 3. Popularity Based Recommendation System

*Author*: Mariam Elsayed

*Contact*: mariamkelsayed@gmail.com

*Notebook*: 3 of 5

*Previous Notebook*: `reviews_loading_preprocessing.ipynb`

*Next Notebook*: `content_rec.ipynb`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Table of Contents

* [Introduction](#intro)

* [Loading the Data](#loading)

* [Popularity-Based Recommendation System](#rec)

    * [Using `average_rating`](#average_rating)

    * [Including a Threshold](#threshold)

    * [Using `rank`](#rank)

* [Conclusion](#conc)

## Introduction <a class="anchor" id="intro"></a>

The first recommendation system we will consider is popularity-based. It is a user-independant recommendation system, meaning that we have no information about the user on what we could make recommendations about. So, we suggest the most popular items.

## Loading the Data <a class="anchor" id="loading"></a>

This recommendation system will use the products data frame with the review summary created in the last notebook.

In [2]:
# loading products dataframe
products_df = pd.read_csv('Data/products_summary.csv')

In [3]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477045 entries, 0 to 477044
Data columns (total 63 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   category                                   477045 non-null  object 
 1   description                                476883 non-null  object 
 2   title                                      477045 non-null  object 
 3   brand                                      477045 non-null  object 
 4   rank                                       477045 non-null  int64  
 5   asin                                       477045 non-null  object 
 6   price                                      477045 non-null  float64
 7   maincat_Luggage & Travel Gear              477045 non-null  bool   
 8   maincat_Backpacks                          477045 non-null  bool   
 9   maincat_Novelty & More                     477045 non-null  bool   
 10  maincat_

## Popularity-Based Recommendation System<a class="anchor" id="rec"></a>

For this recommendation system, we are going to be sorting by highest rating for products with a high number of reviews, and we will also investigating using `rank`.

### Using `average_rating` <a class="anchor" id="average_rating"></a>

Let's start with sorting by the highest average rating.

In [4]:
# Sorting by top 10 products with the highest rating
products_df.sort_values(by=['average_rating'], ascending=False).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
157445,"['Clothing, Shoes & Jewelry', 'Costumes & Acce...",Everyone's favorite chocolate cookie snack is ...,"SAZAC Koala's March Kigurumi (Adults, Yellow)",SAZAC,10216906,B00FFX80RM,88.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
384073,"['Clothing, Shoes & Jewelry', 'Women', 'Jewelr...",Choruslove Sparkling Crystals USA Flag Charms ...,Choruslove Sparkling Crystals USA Flag Charms ...,Choruslove crystal charms,4359613,B017YOLQSY,12.49,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180573,"['Clothing, Shoes & Jewelry', 'Women', 'Jewelr...",.925 Sterling Silver Shiny Dangle Tear Drop Ea...,Ritastephens Sterling Silver Shiny Dangle Tear...,Ritastephens,6117759,B00I0BLGKE,19.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180577,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",Flaunt your chic attitude with the womens Finn...,Finn Comfort Womens Barletta,Finn Comfort,3312796,B00I0BQHLM,327.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180578,"['Clothing, Shoes & Jewelry', 'Women', 'Jewelr...",A brand new ladies 6mm Irish Celtic weave knot...,US Jewels And Gems Ladies 6mm 0.925 Sterling S...,US Jewels And Gems,4118419,B00I0BUN44,46.99,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
384079,"['Clothing, Shoes & Jewelry', 'Women', 'Clothi...",Color block strapless sequin and crepe gown wi...,Jump Juniors' Strapless Crepe and Sequin Long ...,Jump,4659926,B017YQKLEC,37.10,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180581,"['Clothing, Shoes & Jewelry', 'Women', 'Clothi...",Hippie Chic Mesh Cover Up Dress. COQUETA SWIMW...,THE MESH KING Women's Beach Cover-Up Swimsuit ...,THE MESH KING,3190244,B00I0C1HWU,15.97,False,False,False,...,False,False,False,False,False,False,False,False,5.0,3
384078,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",Get ready to move with the beadiee! these whim...,Steve Madden Women's Beadiee Flat Sandal,Unknown,1753274,B017YPZVQG,55.52,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180585,"['Clothing, Shoes & Jewelry', 'Men', 'Jewelry'...",A 14k yellow gold Star of David pendant with a...,US Jewels And Gems New 14k Yellow Gold Star of...,US Jewels And Gems,7178327,B00I0BU09M,272.99,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
180586,"['Clothing, Shoes & Jewelry', 'Novelty & More'...",It's good to run some color bars on your shirt...,Color Bars Test Signal Adult Mens and Womens T...,TooLoud,3367180,B00I0CK294,16.46,False,False,True,...,False,False,False,False,False,False,False,False,5.0,1


The top reviews only have one review with a 5 star rating. This is not the most informative representation of a popular product. 

### Including a Threshold <a class="anchor" id="threshold"></a>

To overcome the issue above, we are going to set a threshold on how many reviews are needed to be considered in the recommendation, then we will recommend the products with the highest reviews.

From the description of the `total_reviews` column, 75% of the products have at least 6 reviews, and 5794 reviews is the highest number of reviews a product has had. Lets pick the threshold to be 500.   

In [5]:
products_df['total_reviews'].describe()

count    477045.000000
mean         12.494433
std          66.232890
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max        5794.000000
Name: total_reviews, dtype: float64

In [6]:
# filtering out products with less than 500 reviews
threshold = 500
products_with_many_reviews = products_df[products_df['total_reviews'] >= threshold]

# sorting by highest average rating
products_with_many_reviews.sort_values(by=['average_rating'], ascending=False).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
201525,"['Clothing, Shoes & Jewelry', 'Shoe, Jewelry &...","', 'GREAT. ANOTHER CRUSHED HEEL.', 'Weve all r...",OrthoStep Metal Shoe Horn 7.5 inch - Made in t...,OrthoStep,3443,B00K0PZ1Y0,10.95,False,False,False,...,False,False,True,False,True,False,False,False,4.9,506
330072,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...",The Only Cruise Tag with a Zip Top Seal Enclos...,Shacke Cruise Tags Zip Seal - Etag Holders & S...,Shacke,21601,B011DDHZJG,10.99,True,False,False,...,False,False,False,False,False,False,True,False,4.8,668
66736,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...","With three large packing cubes included, this ...",eBags Large Packing Cubes for Travel - 3pc Set,Unknown,29099,B005LXPSHG,5.35,True,False,False,...,False,False,False,False,False,False,True,False,4.8,508
144569,"['Clothing, Shoes & Jewelry', 'Shoe, Jewelry &...",Tired Of Bending Over When Putting On Your Sho...,"24"" Inch Extra Long Handled Shoehorn",Shacke,31750,B00E3K9MGQ,12.99,False,False,False,...,False,False,True,False,True,False,False,False,4.8,699
174003,"['Clothing, Shoes & Jewelry', 'Shoe, Jewelry &...",The Perfect Reversible Shoehorn That Fits All ...,"Shacke Metal Shoe Horn 7.5"" inches – Double Si...",Shacke,23409,B00H9RZDRM,5.99,False,False,False,...,False,False,True,False,True,False,False,False,4.8,1254
207618,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...","', 'Pack More Clothes in a Small Space For You...",Shacke Pak - 4 Set Packing Cubes - Travel Orga...,Shacke,118,B00KKXCJQU,24.99,True,False,False,...,False,False,False,False,False,False,True,False,4.8,2406
293785,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",VinicioBelt prides itself in providing timeles...,Men's Holeless Leather Ratchet Click Belt - Tr...,VINICIOBELT,3339,B00VCQ09KW,19.85,False,False,False,...,False,False,False,False,False,False,False,False,4.8,565
292464,"['Clothing, Shoes & Jewelry', 'Boys', 'Jewelry...",Pick your favorite Cross color from menu above...,Horseshoe Nail Cross Necklaces -(Solid Color) ...,Horseshoe Crosses,74357,B00V5RV0ME,29.61,False,False,False,...,False,True,False,False,False,False,False,False,4.8,673
445318,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",*** Perfect Gift! Get it at Today\'s Price! **...,Mt. Eston RFID Blocking Trifold Bifold Mens Wa...,MT. ESTON,4209,B01EDI9U2S,27.99,False,False,False,...,False,False,False,False,False,False,False,False,4.8,590
18712,"['Clothing, Shoes & Jewelry', 'Boys', 'Jewelry...",Pick your favorite Cross color from menu above...,Horseshoe Nail Cross Necklaces -(Solid Color) ...,Horseshoe Crosses,70751,B001DQ5YZ6,52.77,False,False,False,...,False,True,False,False,False,False,False,False,4.8,693


### Using `rank` <a class="anchor" id="rank"></a>

The `rank` column depicts the ranking position of the product in the Clothing, Shoes & Jewelry category determined by Amazon's A9 product search algorithm. This rank is updated once an hour and updated at the same time. Let's use the rank to make a recommendation. 

In [7]:
products_df.sort_values(by=['rank'], ascending=True).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
34139,"['Clothing, Shoes & Jewelry', 'Women', 'Clothi...",A t-shirt bra that won't dig or poke. Warner's...,Warner's Women's This is Not a Bra Full-Covera...,Warner's,2,B00378LDBM,55.03,False,False,False,...,False,False,False,False,False,False,False,False,4.3,34
7668,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",A classic. You'll reach for it every time the ...,Carhartt Men's Acrylic Watch Hat,Unknown,4,B000KGOHLM,12.99,False,False,False,...,False,False,False,False,False,False,False,False,4.7,2238
28976,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",A classic. You'll reach for it every time the ...,Carhartt Men's Acrylic Watch Hat,Unknown,5,B002G9UEG8,12.99,False,False,False,...,False,False,False,False,False,False,False,False,4.6,2264
248633,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,6,B00OSTNWW2,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.1,7
331841,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",Gildan is one of the world's largest verticall...,Gildan Men's Crew T-Shirt Multipack,Unknown,6,B011QQE1HE,39.94,False,False,False,...,False,False,False,False,False,False,False,False,4.4,299
46946,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",The iconic clog that started a comfort revolut...,Crocs Men's and Women's Classic Clog | Comfor...,Unknown,7,B004I6H7SK,90.92,False,False,False,...,False,False,False,False,False,False,False,False,4.3,54
3640,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,7,B000BRETA4,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.4,2249
3642,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,7,B000BRFH1E,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.4,1036
68457,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",The iconic clog that started a comfort revolut...,Crocs Men's and Women's Classic Clog | Comfor...,Unknown,8,B005PQQ1VQ,90.92,False,False,False,...,False,False,False,False,False,False,False,False,4.5,3208
77877,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,9,B006WCLP18,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.6,248


Picking the highest ranks yields different results compared to using the ratings with a threshold. Half of the highest ranks wouldn't meet the threshold set above however the reviews for these products seem mostly good. I believe that the duplicate ranks was caused because the webscraping took place over a longer period of time while the rankings were determined every hours. Since the rankings fluctuate based on Amazon's algorithms, these recommendations could be incorparated to a hybrid model later.

## Conclusion <a class="anchor" id="conc"></a>

A simple popularity-based recommendation system with a threshold was made by considering products with a high number of reviews then by sorting by the average review rating. 

*Next notebook*: `content_rec.ipynb`